In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb


In [2]:
df = pd.read_csv('../input/edsa-climate-change-belief-analysis-2021/train.csv')
df.head()


,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [3]:
fi = pd.read_csv('../input/edsa-climate-change-belief-analysis-2021/test.csv')
fi.head()


,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


In [4]:
df.isnull().sum()

sentiment    0
message      0
tweetid      0
dtype: int64

In [5]:
# write function for removing @user
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i,'',input_txt)
    return input_txt


df['dm'] = np.vectorize(remove_pattern)(df['message'], '@[\w]*')

In [6]:
#removing weblink
df['dm'] = df['dm'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])


In [7]:
# remove special characters, numbers, punctuations
df['dm'] = df['dm'].str.replace('[^a-zA-Z#]+',' ')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [8]:
#removing 1 or 2 leeter words
df['dm'] = df['dm'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))

In [9]:
# create new variable tokenized message 
tokenized_dm = df['dm'].apply(lambda x: x.split())

In [10]:
#stemming the tokenized message
stemmer = SnowballStemmer('english')
tokenized_dm = tokenized_dm.apply(lambda x: [stemmer.stem(i) for i in x])


In [11]:
# join tokens into one sentence
for i in range(len(tokenized_dm)):
    tokenized_dm[i] = ' '.join(tokenized_dm[i])

df['dm']  = tokenized_dm

In [12]:
stopw = stopwords.words('english')
df['dm'].apply(lambda x: [item for item in x if item not in stopw])

0        [p, l, c, j, r,  , e, p,  , c, h, e, f,  , e, ...
1        [n,  , l, k, e,  , l, c, k,  , e, v,  , n, h, ...
2        [r, e, e, r, c, h,  ,  , h, v, e,  , h, r, e, ...
3        [#, n, k, e, r, #,  , w, r, e,  , w,  , p, v, ...
4        [n,  , r, c,  , e, x,  , c, l,  , c, h, n, g, ...
                               ...                        
15814    [h, e,  , k,  , w, n,  , h, e,  , e, r,  , g, ...
15815    [h, w,  , c, l,  , c, h, n, g,  , c, u, l,  , ...
15816    [n, v, e, n,  , n, e, w, r, l,  , w, h,  , e, ...
15817    [h, e,  , l, b, e, r,  , h, e,  , c, l,  , c, ...
15818    [c, l,  , c, h, n, g,  , e, q, u,  , c, r, e, ...
Name: dm, Length: 15819, dtype: object

In [13]:
#split data
X = df['message']
y = df['sentiment']




In [14]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [15]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [16]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [17]:
jaco = fi['message']


In [18]:
#Predictions
fi['sentiment'] = text_clf.predict(jaco)

In [19]:
fi.reset_index()
fi.drop(['message'], axis='columns', inplace=True)

In [20]:
print(fi)

       tweetid  sentiment
0       169760          1
1        35326          1
2       224985          1
3       476263          1
4       872928          0
...        ...        ...
10541   895714          1
10542   875167          1
10543    78329          2
10544   867455          0
10545   470892          1

[10546 rows x 2 columns]


In [21]:
#print(confusion_matrix(y_test,predictions))

In [22]:
#print(classification_report(y_test,predictions))

In [23]:
#print(accuracy_score(y_test,predictions))

In [24]:
fi.to_csv('mycsvfile.csv',index=False)